In [1]:
import pandas as pd
from pathlib import Path
import glob
import sqlite3

In [2]:
conn = sqlite3.connect("mimic_iv.db")

In [ ]:
existing_tables = pd.read_sql_query(
    "SELECT name FROM sqlite_master WHERE type='table';", conn
)['name'].tolist()

folder_hosp = Path("files/mimiciv/3.1/hosp")

for file in folder_hosp.glob("*.csv.gz"):
    table_name = file.stem
    if table_name in existing_tables:
        print(f"Skipping {table_name}, already in database.")
        continue

    try:
        print(f"Inserting {table_name} in chunks...")
        chunks = pd.read_csv(file, compression='gzip', chunksize=50000, low_memory=False)

        for i, chunk in enumerate(chunks):
            chunk.to_sql(table_name, conn, if_exists='append', index=False)
            print(f"Inserted chunk {i + 1}")

    except Exception as e:
        print(f"Failed on {file.name}: {e}")


✅ Skipping admissions.csv, already in database.
✅ Skipping diagnoses_icd.csv, already in database.
✅ Skipping drgcodes.csv, already in database.
✅ Skipping d_hcpcs.csv, already in database.
✅ Skipping d_icd_diagnoses.csv, already in database.
✅ Skipping d_icd_procedures.csv, already in database.
✅ Skipping d_labitems.csv, already in database.
✅ Skipping emar.csv, already in database.
📥 Inserting emar_detail.csv in chunks...
  └─ Inserted chunk 1
  └─ Inserted chunk 2
  └─ Inserted chunk 3
  └─ Inserted chunk 4
  └─ Inserted chunk 5
  └─ Inserted chunk 6
  └─ Inserted chunk 7
  └─ Inserted chunk 8
  └─ Inserted chunk 9
  └─ Inserted chunk 10
  └─ Inserted chunk 11
  └─ Inserted chunk 12
  └─ Inserted chunk 13
  └─ Inserted chunk 14
  └─ Inserted chunk 15
  └─ Inserted chunk 16
  └─ Inserted chunk 17
  └─ Inserted chunk 18
  └─ Inserted chunk 19
  └─ Inserted chunk 20
  └─ Inserted chunk 21
  └─ Inserted chunk 22
  └─ Inserted chunk 23
  └─ Inserted chunk 24
  └─ Inserted chunk 25
  └─ I